In [ ]:
# autoreload
import lightgbm as lgb

from testbed.models.quantile_regression import QuantileRegression
from testbed.models.treeffuser import Treeffuser

import pandas as pd
import numpy as np

from pathlib import Path
from tqdm import tqdm
import pickle as pkl

from testbed.models.ngboost import NGBoostGaussian, NGBoostMixtureGaussian, NGBoostPoisson
from testbed.models.base_model import BayesOptProbabilisticModel


from functools import partial

from jaxtyping import Float, Array
from typing import List, Callable

import seaborn as sns
import matplotlib.pyplot as plt
from testbed.metrics.log_likelihood import LogLikelihoodFromSamplesMetric
from testbed.metrics.crps import CRPS


path = "../src/testbed/data/m5"

# load autoreload extension


In [ ]:
# These are config variables

PROCESS_FROM_SCRATCH = True
USE_SUBSET = True
CONTEXT_LENGTH = 20
RUN_DEPRECATED = False

In [ ]:
# READ IN DATA

sell_prices_df = pd.read_csv(Path(path) / "sell_prices.csv")
sales_train_validation_df = pd.read_csv(Path(path) / "sales_train_validation.csv")
calendar_df = pd.read_csv(Path(path) / "calendar.csv")

print("\ncolumns of sell_prices_df:")
[print(col) for col in sell_prices_df.columns]
print("\ncolumns of sales_train_validation_df:")
[print(col) for col in sales_train_validation_df.columns if not col.startswith("d_")]
print("\ncolumns of calendar_df:") # ommit d_1, d_2, ..., d_1913
[print(col) for col in calendar_df.columns if not col.startswith("d_")]

""

# print number of zeros
print("number of zeros in sales_train_validation_df: ", (sales_train_validation_df == 0).sum().sum())

In [ ]:
#num_zeros = sales_train_validation_df.isin([0]).sum().sum()
#total_entries =   sales_train_validation_df.

items_sold_cols = sales_train_validation_df.columns[sales_train_validation_df.columns.str.startswith("d_")]
num_zeros = (sales_train_validation_df[items_sold_cols] == 0).sum().sum()
total_entries = sales_train_validation_df[items_sold_cols].shape[0] * sales_train_validation_df[items_sold_cols].shape[1]

print(f"number of zeros in sales_train_validation_df: {num_zeros} out of {total_entries} entries")
print(f"percentage of zeros in sales_train_validation_df: {num_zeros / total_entries * 100:.2f}%")

In [ ]:
# add explicit columns for the day, month, year for ease of processing
calendar_df["date"] = pd.to_datetime(calendar_df["date"])
calendar_df["day"] = calendar_df["date"].dt.day
calendar_df["month"] = calendar_df["date"].dt.month
calendar_df["year"] = calendar_df["date"].dt.year


# Brief snapshots of the dataset

In [ ]:
calendar_df.head()

In [ ]:
sales_train_validation_df.head()

In [ ]:
sell_prices_df.head()

# Process the data

In [ ]:
TOTAL_ITEMS = 5000
# select a random subset of items
if USE_SUBSET:
    np.random.seed(0)
    unique_ids = sales_train_validation_df["id"].unique()
    ids = np.random.choice(sales_train_validation_df["id"].unique(), TOTAL_ITEMS, replace=False)
    sales_train_validation_df_sub = sales_train_validation_df[sales_train_validation_df["id"].isin(ids)]
    item_ids = sales_train_validation_df_sub["item_id"].unique()
    sell_prices_df_sub = sell_prices_df[sell_prices_df["item_id"].isin(item_ids)]
    calendar_df_sub = calendar_df





columns_sales_train_validation.head)

The strategy for processing the data is going to be the following. 1) We are going to have X and y where y is the next days sales for a given product. 3) X is made up of 10 previous prices, day of the week, + event types, cat_id, store_id, state_id

In [ ]:
def proc_train_test(sales_train_validation_df: pd.DataFrame, calendar_df: pd.DataFrame, sell_prices_df: pd.DataFrame, context_length: int, test_percentage: float, percentage_omittied: int = 0): #type annotation too long
    """
    This function processes the data and returns the training and test data in two ways:
    - undifferentiated: a list of all training and test data (X_train, y_train, X_test, y_test)
    - differentiated: a list of training and test data for each product (X_train_prod, y_train_prod, X_test_prod, y_test_prod)
        where X_train_prod[i] contains a list of all X_train values for the product i with similar grouping for y_train_prod and test

    This assumes from the dataframes that
    - sales_train_validation_df:
        - has columns with the format d_1, d_2, ...
        - has columns item_id and store_id
    - calendar_df:
        - wday, month, event_name_1, event_name_2
    - sell_prices_df:
        - item_id, store_id, sell_price

    - percentage_omittied: percentage of the data to be omitted from the training data and the test data
        (randomly selected)

    Returns:
    - undifferentiated: Tuple of X_train, y_train, X_test, y_test
    - differentiated: Tuple of X_train_prod, y_train_prod, X_test_prod, y_test_prod
    """
    np.random.seed(0)
    # First we need to get the training data
    # We will use the first 1913 days as training data and the next

    X_train = []
    y_train = []

    X_test = []
    y_test = []

    # We will also return a second grouping of lists where X_train_prod[i] contains a
    # a list of all X_train values for the product i with similar grouping for y_train_prod and test
    X_train_prod = []
    y_train_prod = []
    X_test_prod = []
    y_test_prod = []


    # get all days that start with d_ and look for the maximum
    total_days = max([int(x.split("_")[1]) for x in sales_train_validation_df.columns if "d_" in x])
    train_days = int(total_days * (1 - test_percentage))
    print("train days", train_days)
    print("test days", total_days - train_days)
    print("total days", total_days)

    # Precompute the required data
    calendar_df_dict = calendar_df.set_index("d").to_dict(orient="index")
    sell_prices_dict = sell_prices_df.groupby(["item_id", "store_id"])["sell_price"].first().to_dict()

    pbar = tqdm(total=len(sales_train_validation_df))
    for _, row in sales_train_validation_df.iterrows():
        item_id = row["item_id"]
        store_id = row["store_id"]

        X_train_prod.append([])
        y_train_prod.append([])
        X_test_prod.append([])
        y_test_prod.append([])

        pbar.update(1)

        valid_size = int((train_days - context_length) * (1 - percentage_omittied))
        valid_js = np.random.choice(range(1, train_days - context_length), valid_size, replace=False)

        valid_js = list(valid_js) + list(range(train_days, total_days - context_length))

        for j in valid_js:
            x = []

            # Add sales values for the previous context_length days
            x.extend(row[f"d_{j+k}"] for k in range(context_length))

            # Add additional features
            current_day = f"d_{j+context_length}"
            calendar_data = calendar_df_dict[current_day]
            x.extend([
                calendar_data["wday"],
                calendar_data["month"],
                store_id,
                calendar_data["event_name_1"],
                calendar_data["event_name_2"],
                sell_prices_dict[(item_id, store_id)],
                item_id
            ])

            if j < train_days:
                X_train.append(x)
                y_train.append(row[current_day])
                X_train_prod[-1].append(x)
                y_train_prod[-1].append(row[current_day])

            else:
                X_test.append(x)
                y_test.append(row[current_day])
                X_train_prod[-1].append(x)
                y_train_prod[-1].append(row[current_day])

    undifferentiated = (X_train, y_train, X_test, y_test)
    differentiated = (X_train_prod, y_train_prod, X_test_prod, y_test_prod)
    return undifferentiated, differentiated

In [ ]:
if PROCESS_FROM_SCRATCH:
    undifferentiated, differentiated = proc_train_test(sales_train_validation_df_sub, calendar_df, sell_prices_df_sub, CONTEXT_LENGTH, 0.02, 0.99)
    X_train, y_train, X_test, y_test = undifferentiated
    X_train_prod, y_train_prod, X_test_prod, y_test_prod = differentiated


In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
COL_NAMES = [
    f"day_{i}" for i in range(1, CONTEXT_LENGTH+1)
] + ["wday", "month", "store_id", "event_name_1", "event_name_2", "sell_price", "item_id"]

CAT_COLS = ["store_id", "event_name_1", "event_name_2", "item_id", "wday", "month"]
CAT_COLS_IDX = [COL_NAMES.index(col) for col in CAT_COLS]


In [ ]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)
y_train_df = pd.DataFrame(y_train)

X_train_df.columns = COL_NAMES
X_test_df.columns = COL_NAMES

In [ ]:

X_train_df

In [ ]:
# Encode the categorical columns as numbers
from sklearn.preprocessing import LabelEncoder
# Get only label of item_id
X_train_df["item_id"] = X_train_df["item_id"].apply(lambda x: x.split("_")[1])
X_test_df["item_id"] = X_test_df["item_id"].apply(lambda x: x.split("_")[1])


label_encoders = {}
for col in CAT_COLS:
    le = LabelEncoder()
    X_train_df[col] = le.fit_transform(X_train_df[col])
    X_test_df[col] = le.transform(X_test_df[col])
    label_encoders[col] = le


X_train_prod_processed = []
X_test_prod_processed = []
for i in range(len(X_train_prod)):
    X_train_prod_processed.append(pd.DataFrame(X_train_prod[i], columns=COL_NAMES))
    X_test_prod_processed.append(pd.DataFrame(X_test_prod[i], columns=COL_NAMES))
    X_train_prod_processed[-1]["item_id"] = X_train_prod_processed[-1]["item_id"].apply(lambda x: x.split("_")[1])
    X_test_prod_processed[-1]["item_id"] = X_test_prod_processed[-1]["item_id"].apply(lambda x: x.split("_")[1])
    for col in CAT_COLS:
        X_train_prod_processed[-1][col] = label_encoders[col].transform(X_train_prod_processed[-1][col])
        X_test_prod_processed[-1][col] = label_encoders[col].transform(X_test_prod_processed[-1][col])

X_train_df.head()


# PPC

### "Standard PPCs"

In [ ]:

def max_ppc(y_true: Float[Array, "batch y_dim"], y_samples: Float[Array, "samples batch y_dim"], number=0, name="") -> None:
    # rpeat y_true to match the shape of y_samples
    max_ppc = np.max(y_samples, axis=1)
    true_max = np.max(y_true)

    return max_ppc.flatten(), true_max.flatten(), "max_ppc"

def quantile_ppc(y_true: Float[Array, "batch y_dim"], y_samples: Float[Array, "samples batch y_dim"], quantile=0.5, number=0, name="") -> None:
    # rpeat y_true to match the shape of y_samples
    q = np.quantile(y_samples, quantile, axis=1)
    true_q = np.quantile(y_true, quantile)
    return q.flatten(), true_q.flatten(), f"quantile_ppc_{quantile}"

def zeros(y_true: Float[Array, "batch y_dim"], y_samples: Float[Array, "samples batch y_dim"], number=0, name="") -> None:
    "Count the number of zeros in the samples"
    zeros = np.sum(y_samples < 0.1, axis=1)
    true_zeros = np.sum(y_true < 0.1)

    return zeros.flatten(), true_zeros.flatten(), "zeros"

def percentage_zeros(y_true: Float[Array, "batch y_dim"], y_samples: Float[Array, "samples batch y_dim"], number=0, name="") -> None:
    "Count the number of zeros in the samples"
    zeros = np.mean(y_samples < 0.1, axis=1)
    true_zeros = np.mean(y_true < 0.1)

    return zeros.flatten(), true_zeros.flatten(), "percentage_zeros"

In [ ]:
def plot_ppcs(y_true: Float[Array, "batch y_dim"], y_samples: Float[Array, "samples batch y_dim"], ppcs: List[Callable],
              number=0, name="") -> None:
    # plot the distribution of

    for ppc in ppcs:
        ppc(y_true, y_samples, number=number, name=name)

### "Complex PPCs"

In [ ]:

def plot_model_comparisons(data, y_true, figsize=(12, 8), model_names=None):
    """
    Plots model predictions against true values for each day.

    :param data: numpy array of shape [models, samples, days] containing model predictions
    :param y_true: array of shape [days] containing the true values
    :param figsize: tuple indicating the size of the figure
    """
    sns.set(style="whitegrid")
    models, samples, days = data.shape

    # Create a figure and axis object
    fig, ax = plt.subplots(figsize=figsize)

    # We will transform the data to a format suitable for seaborn
    # Create a DataFrame with model, day, and sample values
    plot_data = []
    if model_names is None:
        model_names = [f"Model {i}" for i in range(models)]

    for model_idx in range(models):
        for day_idx in range(days):
            for sample_idx in range(samples):
                plot_data.append({
                    "Day": day_idx,
                    "Value": data[model_idx, sample_idx, day_idx],
                    "Model": model_names[model_idx]
                })

    import pandas as pd
    plot_data = pd.DataFrame(plot_data)

    # Use seaborn to plot the boxplots
    sns.boxplot(x="Day", y="Value", hue="Model", data=plot_data, ax=ax, width=0.6)

    # Plot true values
    plt.plot(y_true, 'o', color='red', label='True Values')

    # Setting labels and title
    plt.xticks(ticks=np.arange(days), labels=[f"Day {i+1}" for i in range(days)])
    plt.xlabel('Days')
    plt.ylabel('Values')
    plt.title('Model Predictions vs. True Values')
    plt.legend()

    # Show the plot
    plt.show()







# Model Evaluation

In [ ]:
def save_results_to_pkl(results: dict, dir_name, name):
    if not Path(dir_name).exists():
        Path(dir_name).mkdir(parents=True)

    path = Path(dir_name) / f"{name}.pkl"
    with open(path, "wb") as f:
        pkl.dump(results, f)


def load_results_from_pkl(dir_name, name):
    path = Path(dir_name) / f"{name}.pkl"
    with open(path, "rb") as f:
        results = pkl.load(f)
    return results


In [ ]:
# Simple helper function to train a model and plot ppcs

def get_ppcs(y_samples, X_test, y_test, ppcs, number=0, name="") -> None:
    """
    Returns a dictionary with the samples and the true values for each ppc
    the dictionary a
    """
    y_samples = np.array(y_samples)
    #y_samples = np.maximum(y_samples, 0)
    #y_samples = np.round(y_samples, 0)

    ppc_results = {}
    for ppc in ppcs:
        samples, true, name = ppc(y_test, y_samples, number=number, name=name)
        ppc_results[name] = {"samples": samples, "true": true}

    return ppc_results


In [ ]:
print(X_train_df.head())

In [ ]:
EVAL_VALUES = 10_000 #len(X_test_df)
np.random.seed(0)

eval_idx = np.random.choice(len(X_test_df), EVAL_VALUES, replace=False)

X_train_np = X_train_df.values
X_test_np = X_test_df.values[eval_idx]

y_train_np = y_train_df.values + np.random.normal(0, 0.01, y_train_df.shape)
y_test_np = y_test_df.values[eval_idx]

# change to float to prevent errors
y_train_np = y_train_np.astype(np.float32)
y_test_np = y_test_np.astype(np.float32)

dataset = {
    "X_train": X_train_np,
    "X_test": X_test_np,
    "y_train": y_train_np,
    "y_test": y_test_np,
    "col_names": COL_NAMES,
    "cat_cols": CAT_COLS,
}

with open("dataset.pkl", "wb") as f:
    pkl.dump(dataset, f)

In [ ]:
MODEL_CLASSES = [NGBoostPoisson, Treeffuser]
NAMES = ["NGBoostPoisson", "Treeffuser"]

#MODEL_CLASSES = MODEL_CLASSES[-1:]
#NAMES = NAMES[-1:]

NUM_SAMPLES = 100
HYPERS = [
    {"subsample": 0.20, "subsample_freq": 1, "verbose": 0, "num_leaves":129, "learning_rate":0.5,
     "sde_manual_hyperparams": {"hyperparam_max": 10}},
    {},
    {}
]

results = []
for i in range(len(MODEL_CLASSES)):
    model_cls = MODEL_CLASSES[i]
    model = BayesOptProbabilisticModel(model_cls, n_iter_bayes_opt=20, frac_validation=0.01)
    #model = model_cls(**HYPERS[i])


    if model_cls == NGBoostPoisson:
        # shuffle the data
        np.random.seed(0)
        idx = np.random.permutation(len(X_train_np))
        X_train_np_ngb = X_train_np[idx]
        y_train_np_ngb = y_train_np[idx].astype(np.int32)
        model.fit(X_train_np_ngb, y_train_np_ngb)

    elif model_cls == NGBoostGaussian:
        y_train_np_ngb = y_train_np + np.random.normal(0, 3, y_train_np.shape)
        # rescale
        #y_train_np_ngb = (y_train_np_ngb - np.mean(y_train_np_ngb)) / np.std(y_train_np_ngb)
        model.fit(X_train_np, y_train_np_ngb)

    else:
        model.fit(X_train_np, y_train_np)

    results.append({
        "model": model,
        "model_name": NAMES[i]
    })

    save_results_to_pkl(results, "m5", "results.pkl")


results = load_results_from_pkl("m5", "results.pkl")


# Save the results


In [ ]:

for i, result in enumerate(results):
    model = result["model"]
    model_name = result["model_name"]
    print(f"Model: {model_name}")
    y_samples = model.sample(X_test_np, NUM_SAMPLES)
    results[i]["y_samples"] = y_samples

In [ ]:
results_no_model = []
for result in results:
    results_no_model.append({k: v for k, v in result.items() if k != "model"})

# Don't uncomment or will overwrite the results
#save_results_to_pkl(results, "m5", "results_final.pkl")
#save_results_to_pkl(results_no_model, "m5", "results_no_model_final.pkl")

In [ ]:
results_no_model = load_results_from_pkl("m5", "results_final.pkl")

Now we can actually fit some of the models

In [ ]:
ppcs = [max_ppc, zeros, percentage_zeros] + [partial(quantile_ppc, quantile=q) for q in [0.1, 0.5, 0.9, 0.99, 0.999]]

for i, model_cls in enumerate(MODEL_CLASSES):
    ppc_results = get_ppcs(
        y_samples=results_no_model[i]["y_samples"],
        X_test=X_test_np,
        y_test=y_test_np,
        ppcs=ppcs,
        number=i,
        name=model_cls.__name__
    )
    results[i]["ppc_results"] = ppc_results


# Plot the PPCs

In [ ]:
# Titles for plots
ppc_tiles = {
     "max_ppc": "$\max$",
      "zeros": r"$\text{zeros}$",
      "quantile_ppc_0.99": "$q_{0.99}$",
 }

In [ ]:
def set_plot_style():
    """
    Sets a common plotting style for all of the figures that will be
    used in the final paper.
    """

    # no grid but white with pretty ticks


    # use latex font by default
    plt.rc("text", usetex=False)
    plt.rc("font", family="serif")
    sns.set_style("white")

    # make it ready for a presentation
    sns.set_context("talk")
set_plot_style()

In [ ]:
def proc_title(title):
    if title in ppc_tiles:
        print("title", title)
        return ppc_tiles[title]

    x = title.replace("_", " ").capitalize()
    x = x.replace("ppc", "")
    return x

ppc_number = len(ppcs)
ppc_names = results[0]["ppc_results"].keys()

for ppc_name in ppc_names:
    fig, ax = plt.subplots()
    for i, res in enumerate(results):
        model_name = res["model_name"]
        samples = res["ppc_results"][ppc_name]["samples"]
        # make int
        samples = np.maximum(samples, 0)
        samples =  np.round(samples)
        true = res["ppc_results"][ppc_name]["true"]
        n_unique_samples = len(np.unique(samples))
        discrete = n_unique_samples < 20

        print("samples", samples)

        # plot a histogram of the samples but with integers (use nice binning)
        if ppc_name == "max_ppc":
            binwidth = 70
        else:
            binwidth = None
        sns.histplot(samples, ax=ax, label=f"{model_name}" + r" $\hat{p}$", discrete=discrete, stat="density", binwidth=binwidth)
        if i == 0:
            ax.axvline(true, color="red", label="Observed Value")
        ax.set_title(proc_title(f"{ppc_name}"))
        ax.legend()

        if n_unique_samples < 2:
            min_val = np.min(samples) - 1
            max_val = np.max(samples) + 1

            ax.set_xlim(min_val, max_val)



        # save the figure
        fig.savefig(f"m5/{ppc_name}.png", dpi=100)
        # save as pdf
        fig.savefig(f"m5/{ppc_name}.pdf")

        #max_x = true * 5
        #max_samples = np.max(samples)
        #if max_samples > max_x:
        #    ax.set_xlim(0, max_x)


    plt.show()





In [ ]:
for i, result in enumerate(results):
    samples = result["y_samples"][0]
    samples = np.round(samples) + (i+1)/4
    sns.histplot(samples.flatten(), stat="density", label=result["model_name"])

sns.histplot(y_test_np.flatten(), stat="density", color="red", label="true")
plt.xlim(0, 10)
plt.legend()


In [ ]:
np.random.seed(0)

NUM_PRODS_TO_PLOT = 3
DAYS_TO_PLOT = 10
QUATILE = 0.9
NUM_SAMPLES = 100

prods_to_plot = np.random.choice(range(len(X_train_prod_processed)), NUM_PRODS_TO_PLOT)

means_to_plot = []
lower_q_s = []
upper_q_s = []

prod_dict = {}

for res in results:
    model = res["model"]

    for i in prods_to_plot:
        if i not in prod_dict:
            prod_dict[i] = {
                "means_to_plot": [],
                "lower_q_s": [],
                "upper_q_s": [],
                "samples": []
            }

        X_prod_proc_i = X_train_prod_processed[i][-DAYS_TO_PLOT:]
        y_prod_proc_i = y_train_prod[i][-DAYS_TO_PLOT:]

        samples = model.sample(X_prod_proc_i.values, NUM_SAMPLES)
        samples = samples.astype(int)
        samples = np.maximum(samples, 0)

        means = np.mean(samples, axis=0)
        lower_q = np.quantile(samples, 1-QUATILE, axis=0)
        upper_q = np.quantile(samples, QUATILE, axis=0)

        prod_dict[i]["means_to_plot"].append(means)
        prod_dict[i]["lower_q_s"].append(lower_q)
        prod_dict[i]["upper_q_s"].append(upper_q)
        prod_dict[i]["samples"].append(samples)

for i in prod_dict:
    means_to_plot = np.array(prod_dict[i]["means_to_plot"]).squeeze()
    lower_q_s = np.array(prod_dict[i]["lower_q_s"]).squeeze()
    upper_q_s = np.array(prod_dict[i]["upper_q_s"]).squeeze()
    samples = np.array(prod_dict[i]["samples"]).squeeze()
    y_true = np.array(y_train_prod[i][-DAYS_TO_PLOT:])

    model_names = [res["model"].__class__.__name__ for res in results]

    print("mean shape", means_to_plot.shape)
    print("lower shape", lower_q_s.shape)
    print("upper shape", upper_q_s.shape)
    print("y_true shape", y_true.shape)
    print("samples shape", samples.shape)

    #plot_predictions(y_true, means_to_plot, upper_q_s, lower_q_s, [res["model"].__class__.__name__ for res in results])
    plot_model_comparisons(samples, y_true, model_names=model_names)


# Log-likelihood

In [ ]:
for res in results:
    model = res["model"]
    name = res["model_name"]
    nll = LogLikelihoodFromSamplesMetric(n_samples=100).compute(model=model, X_test=X_test_np, y_test=y_test_np, samples=res["y_samples"])
    crps  = CRPS().compute(model=model, X_test=X_test_np, y_test=y_test_np, samples=res["y_samples"])
    print(f"Model {name} has a NLL of {nll}")
    print(f"Model {name} has a CRPS of {crps}")

# Plot calibration plot

In [ ]:
def calibration_plot(y_samples: Float[np.ndarray, "n_samples batch y_dim"], y_test: Float[np.ndarray, "batch y_dim"]) -> None:
    """
    We will plot the calibration plot for the model. Essentially, we will plot the
    """
    assert y_test.shape[1] == 1, "Only works for univariate outputs"
    n_samples = y_samples.shape[0]
    y_samples = np.maximum(y_samples, 0.0)
    y_samples = np.round(y_samples).astype(int)
    y_test = y_test.astype(int)

    # Filter out the zeros
    #non_zero_idx = y_test > 0
    #y_test = y_test[non_zero_idx]
    #y_samples = y_samples[:, non_zero_idx]

    y_test_expanded = y_test[np.newaxis, :].repeat(n_samples, axis=0)
    prob_of_event = np.mean(y_samples <= y_test_expanded, axis=0)
    prob_of_event_sorted = np.sort(prob_of_event.flatten())
    return np.linspace(0, 1, len(prob_of_event)), prob_of_event_sorted










In [ ]:
y_ex = None
for res in results:
    samples = res["y_samples"]
    model = res["model"]
    x,y = calibration_plot(samples, y_test_np)
    plt.plot(x, y, label=res["model_name"])
    y_ex = y


plt.plot([0, 1], [0, 1], linestyle="--", color="black")

plt.xlabel("Predicted Probability")
plt.ylabel("True Probability")
plt.legend()

plt.show()

# plot the distribution of y

sns.histplot(y_ex, bins=20)

In [ ]:
# Quantile prediction plot for the model
def make_quantile_plot(results):
    quantiles = np.linspace(0.9, 0.999, 10)

    means = []
    stds = []

    for res in results:
        samples = res["y_samples"] # shape [n_samples, batch, y_dim]
        samples = np.maximum(samples, 0)
        samples = np.round(samples)

        m, s = [], []

        for q in quantiles:
            quantile_samples = np.quantile(samples, q, axis=1)
            mean = np.mean(quantile_samples, axis=0)
            std = np.std(quantile_samples, axis=0)
            m.append(mean)
            s.append(std)

        m = np.array(m).squeeze()
        s = np.array(s).squeeze()
        plt.plot(quantiles, m, label=res["model_name"])
        plt.fill_between(quantiles, m - s, m + s, alpha=0.3)

        means.append(m)
        stds.append(s)


    true_quantiles =  []
    for q in quantiles:
        true_quantiles.append(np.quantile(y_test_np, q))

    plt.plot(quantiles, true_quantiles, label="True")

    # set log scale on y axis


    # set log scale on x axis
    plt.xscale("log")
    plt.legend()



In [ ]:
make_quantile_plot(results)